# VTK Output for 3D (and 2D) Visualization

Besides the convenient visualizations of the Netgen GUI and the NGSolve webgui, we can also use the VTK output for 3D visualization. 

This is particularly useful for polished visualizations of 3D simulations where you want to apply several filters to the visualization, e.g. to show the solution on a slice of the domain, or to show the solution as a surface plot and possibly combine these things. 

## Setup dependencies

In this unit we will use `pyvista` to visualize the VTK output and we will need `vtk` as well. You can install these packages via pip in your terminal (or a virtual env.):

```bash
pip3 install pyvista vtk ipywidgets trame trame-vtk trame-vuetify --break-system-packages --upgrade
```
or execute the following cell:

In [1]:
!PIP_BREAK_SYSTEM_PACKAGES=1 pip3 install pyvista vtk ipywidgets trame trame-vtk trame-vuetify --upgrade

## Functionality on dummy data

If you want to, you can skip to the next section for the visualization of a flow field.

### Generating the VTK output

In [2]:
from ngsolve import *
from netgen.occ import unit_cube

In [3]:
#VTKOutput?

In [4]:
mesh = Mesh (unit_cube.GenerateMesh(maxh=0.4))
gfu = GridFunction (L2(mesh=mesh, order=3))
gfu.Set(sin(4*pi*x*y*z))
gfv = GridFunction (L2(mesh=mesh, order=3))
gfv.Set(cos(x**2+y**2-z**2))

#define VTK output for visualization:
vtk = VTKOutput(mesh,coefs=[gfu,gfv],names=["gfu","gfv"],filename="vtkout",subdivision=3, floatsize="single",legacy=False)
#execute the VTK output:
vtk.Do()

'vtkout'

### Reading the VTK data to `pyvista`

In [5]:
import pyvista
pyvista.set_jupyter_backend('html')
visobj = pyvista.read('vtkout.vtu')

### Simple plotting

Option 1: Call `plot`: (will give no result for jupyter book)

In [ ]:
visobj.plot(scalars="gfu")

KeyError: 'Data array (u) not present in this dataset.'

Option 1b: Call `plot` with several options:  (will give no result for jupyter book)

In [7]:
visobj.plot(scalars="gfu", show_scalar_bar=True, show_edges=False, 
            edge_color='black', cmap='jet', lighting=True, 
            n_colors=16, clim=[-1,1], 
            scalar_bar_args={'title':'u', 'height':1.5, 'width':0.5, 
                             'position_x':0.1, 'position_y':0.1})

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Option 2: Use a visualization object (`pyvista.Plotter`) and add the data (possibly step by step) to it:

In [8]:
plot = pyvista.Plotter()
plot.add_mesh(visobj, scalars="gfu", cmap="terrain")
plot.show()
plot.export_html("plot1.html")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

For jupyter book take a look [here](plot1.html)

### Some filters

Slices: (will give no result for jupyter book)

In [9]:
orthoslices = visobj.slice_orthogonal()
orthoslices.plot(scalars="gfu")

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Iso-surfaces: (will give no result for jupyter book)

In [10]:
contour = visobj.contour([0.25], scalars="gfu", rng=[-1,1])
print(contour)
contour.plot(color="lightblue", smooth_shading=False)

PolyData (0x72d6e6153e80)
  N Cells:    5709
  N Points:   4479
  N Strips:   0
  X Bounds:   2.164e-02, 1.000e+00
  Y Bounds:   2.164e-02, 1.000e+00
  Z Bounds:   2.168e-02, 1.000e+00
  N Arrays:   2


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

Combining stuff: (will give no result for jupyter book)

In [11]:
import numpy as np
def translation_matrix(x, y, z):
    return np.array([
        [1, 0, 0, x],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])
box_moved_left = visobj.copy().transform(translation_matrix(-1,0,0))
orthoslices_moved_right = orthoslices.combine().transform(translation_matrix(1,0,0))

plot = pyvista.Plotter(shape=(1,2))
plot.subplot(0,0)
plot.add_text("subplot1", font_size=24)
plot.add_mesh(box_moved_left, scalars="gfu", cmap="jet")
plot.add_mesh(contour, color="lightblue", opacity=0.7)
plot.add_mesh(orthoslices_moved_right, scalars="gfu", cmap="jet")
plot.subplot(0,1)
plot.add_text("subplot2", font_size=24)
plot.add_mesh(visobj, scalars="gfv")

plot.show()
plot.export_html("plot2.html")

/home/crist/.venvs/fcenv/lib/python3.12/site-packages/pyvista/core/filters/data_object.py:180: PyVistaDeprecationWarning: The default value of `inplace` for the filter `UnstructuredGrid.transform` will change in the future. Previously it defaulted to `True`, but will change to `False`. Explicitly set `inplace` to `True` or `False` to silence this warning.
  warnings.warn(msg, PyVistaDeprecationWarning)


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

For jupyter book take a look [here](plot2.html)

## Visualizing for the flow problem

We assume that you ran the [`navierstokes_hdg_chorin.ipynb` example](navierstokes_hdg_chorin.ipynb) successfully (in parallel) and have the VTK output files available.

In [12]:
import pyvista
pyvista.set_jupyter_backend('html')
visobj = pyvista.read('parstokes.pvd')

In [13]:
visobj = visobj.combine()

(next cell generates no output in jupyter book)

In [14]:
orthoslices = visobj.slice_orthogonal().combine()
orthoslices.plot(scalars="velocity",cmap="jet",clim=[0,2.25])
orthoslices.plot(scalars="pressure",cmap="coolwarm",clim=[-0.06,0])

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

In [15]:
plot = pyvista.Plotter()
arrows = orthoslices.glyph(scale="velocity", orient="velocity", tolerance=0.03)

seed = pyvista.Box(bounds=(-2,1,-1,1,-1,1), level=5)

strl = visobj.streamlines_from_source(
    seed,
    vectors="velocity",
    max_step_length=1,
    min_step_length=0.1,
    max_time=500,
    max_steps=5000,
    max_error=1e-6,
    integration_direction="forward",
)

plot.add_mesh(orthoslices, scalars="velocity", cmap="jet")
#plot.add_mesh(arrows, color="black")
plot.add_mesh(strl, color="black")
#plot.add_mesh(pyvista.Cylinder(center=(0.5,0.2,0.2), direction=(0,0,1), height=0.41, radius=0.05), color="white")
plot.show()
plot.export_html("plot5.html")

/home/crist/.venvs/fcenv/lib/python3.12/site-packages/pyvista/core/filters/data_set.py:3204: PyVistaDeprecationWarning: ``max_time`` parameter is deprecated.  It will be removed in v0.48
  warnings.warn(


EmbeddableWidget(value='<iframe srcdoc="<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv=&quot;Content-…

When considered in html export (jupyter book), take a look [here](plot5.html)